In [2]:
import openai

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-03-15-preview",
    temperature = 0.2, 
    model = "gpt-4o-mini"
    )

/home/varun/Documents/MS/GenAI/venv_gen/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [12]:
messages = [
    ("system","You're a helpful assistant that tranlates English to Spanish."),
    ("user","Hello ! Can you please help me, I'm new here !")
]

ai_msg = llm.invoke(messages)
ai_msg.content

'¡Hola! Claro, estaré encantado de ayudarte. ¿En qué necesitas ayuda?'

In [34]:
# Together AI
from langchain_together import ChatTogether

llm = ChatTogether(
    api_key=os.getenv("TOGETHER_API_KEY"),
    temperature= 0.2,
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    max_tokens=200
)

llm

ChatTogether(client=<openai.resources.chat.completions.Completions object at 0x76b62836cb80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x76b61037d8a0>, model_name='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', temperature=0.2, openai_api_key=SecretStr('**********'), max_tokens=200, together_api_key=SecretStr('**********'))

In [35]:
messages = [
    ("system", "You're a helpful assistant that tranlates English to French."),
    ("human", "Hello ! Can you please help me, I'm new here !")
]

ai_msg = llm.invoke(messages)
ai_msg.content

"Bonjour ! Bien sûr, je suis là pour vous aider ! Comment puis-je vous aider aujourd'hui ? (Hello! Of course, I'm here to help! How can I assist you today?)"

In [36]:
# Prompt Template
from langchain import LLMChain, PromptTemplate

#define the promt template
product_description_template = PromptTemplate(
    input_variable = ["product_name", "product_features"],
    template = ''' 
    You are a creative copywriter. Write an engaging product description for the following product:

    Product Name: {product_name}
    Features: {product_features}

    The description should be catchy and highlight the unique selling points of the product.
    '''
)

In [37]:
def generate_product_derc(product_name, product_feature):
    
    chain = LLMChain(
        llm = llm, 
        prompt = product_description_template
        )
    
    response = chain.invoke(
        {
            "product_name": product_name,
            "product_features" :product_feature
        }
    )
    return response

In [38]:
product_name = "Smart Home Design"
product_feature = "Voice control, AI-powered, Energy efficient, Multi-room support"

generate_product_derc(product_name, product_feature)

{'product_name': 'Smart Home Design',
 'product_features': 'Voice control, AI-powered, Energy efficient, Multi-room support',
 'text': "**Introducing Smart Home Design: Where Innovation Meets Elegance**\n\nImagine a home that adapts to your every need, where the lights dim to perfection as you settle in for a movie night, and the thermostat adjusts to your preferred temperature without you even lifting a finger. Welcome to the future of home living with Smart Home Design.\n\nThis revolutionary system is not just a smart home device - it's a personal assistant, a energy manager, and a design expert all in one. With voice control at your fingertips (or rather, your voice), you can effortlessly manage your home's ambiance, security, and energy efficiency.\n\n**The Power of AI**\n\nOur AI-powered technology learns your habits and preferences, anticipating your needs and adjusting the settings accordingly. Want to wake up to a bright and cheerful morning? Smart Home Design will have the lig

In [49]:
# Chain Prompt Template
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_template(
'''
Translate the following review to English.
{Review}
'''
)

chain_one = LLMChain(llm=llm,
                     prompt = first_prompt,
                     output_key = "English_Review"
    )

In [50]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

chain_two = LLMChain(llm=llm, 
                     prompt=second_prompt,
                     output_key="summary"
                    )

In [51]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review, answer with just the name of the language:\n\n{Review}"
)

chain_three = LLMChain(llm=llm, 
                       prompt=third_prompt,
                       output_key="language"
                      )

In [52]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment of the following summary and output it in any of these: (positive, negative, neutral):\n\nSummary: {summary}"
)
chain_four = LLMChain(llm=llm, 
                      prompt=fourth_prompt, 
                      output_key="sentiment")

In [59]:
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables = ["English_Review","summary", "language", "sentiment"],
    verbose = True
)

In [60]:
# Run the SequentialChain
review = 'एसर का नया एडवांस आई-सीरीज 43-इंच टीवी देखने में जितना आकर्षक है, इस्तेमाल करने में उतना ही स्मूथ। इसकी पिक्चर क्वालिटी और साउंड दोनों ही शानदार हैं। स्मार्ट फीचर्स के मामले में भी ये टीवी पीछे नहीं है, गूगल असिस्टेंट सपोर्ट के साथ आप आराम से वॉइस कमांड दे सकते हैं। हालांकि, हैंड्स-फ्री फीचर की कमी थोड़ी खलती है। इसके अलावा, टीवी के साथ दिया गया रिमोट भी काफी अच्छा है, सभी जरूरी बटन और OTT प्लेटफॉर्म के लिए डेडिकेटेड बटन भी मौजूद हैं। कुल मिलाकर, अगर आप एक अच्छा और किफायती स्मार्ट टीवी ढूंढ रहे हैं तो एसर का ये टीवी आपके लिए एक बढ़िया विकल्प हो सकता है।'
result = overall_chain({"Review": review})
print(result)



> Entering new SequentialChain chain...

> Finished chain.
{'Review': 'एसर का नया एडवांस आई-सीरीज 43-इंच टीवी देखने में जितना आकर्षक है, इस्तेमाल करने में उतना ही स्मूथ। इसकी पिक्चर क्वालिटी और साउंड दोनों ही शानदार हैं। स्मार्ट फीचर्स के मामले में भी ये टीवी पीछे नहीं है, गूगल असिस्टेंट सपोर्ट के साथ आप आराम से वॉइस कमांड दे सकते हैं। हालांकि, हैंड्स-फ्री फीचर की कमी थोड़ी खलती है। इसके अलावा, टीवी के साथ दिया गया रिमोट भी काफी अच्छा है, सभी जरूरी बटन और OTT प्लेटफॉर्म के लिए डेडिकेटेड बटन भी मौजूद हैं। कुल मिलाकर, अगर आप एक अच्छा और किफायती स्मार्ट टीवी ढूंढ रहे हैं तो एसर का ये टीवी आपके लिए एक बढ़िया विकल्प हो सकता है।', 'English_Review': "Here is the translation of the review to English:\n\nAsus's new Advance i-Series 43-inch TV is as attractive to watch as it is to use. Its picture quality and sound are both excellent. In terms of smart features, this TV is not behind either, with Google Assistant support, you can easily give voice commands. However, the lack of a hands-free fe

In [61]:
# Router Chain

In [62]:
from langchain.chains import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [64]:
positive_feedback_template = """You are an empathetic customer service representative.
Summarize the following positive feedback and generate a thank-you response.
Here is the feedback:
{input}"""

negative_feedback_template = """You are an empathetic customer service representative.
Summarize the following negative feedback and generate an apology response with a follow-up question for more details.
Here is the feedback:
{input}"""

neutral_feedback_template = """You are an empathetic customer service representative.
Summarize the following neutral feedback and generate a generic response.
Here is the feedback:
{input}"""


prompt_infos = [
    {
        "name": "positive",
        "description": "Good for processing positive feedback",
        "prompt_template": positive_feedback_template
    },
    {
        "name": "negative",
        "description": "Good for processing negative feedback",
        "prompt_template": negative_feedback_template
    },
    {
        "name": "neutral",
        "description": "Good for processing neutral feedback",
        "prompt_template": neutral_feedback_template
    }
]

In [66]:
# Destination Chains Setup: A loop iterates over each item in prompt_infos,
# creates a chain for each feedback type,
# and stores it in the destination_chains dictionary.
# Each chain is created by combining the ChatPromptTemplate with LLMChain.


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain # this is what we'll pass to the router chain

destination_chains

{'positive': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an empathetic customer service representative.\nSummarize the following positive feedback and generate a thank-you response.\nHere is the feedback:\n{input}'))]), llm=ChatTogether(client=<openai.resources.chat.completions.Completions object at 0x76b62836cb80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x76b61037d8a0>, model_name='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', temperature=0.2, openai_api_key=SecretStr('**********'), max_tokens=200, together_api_key=SecretStr('**********'))),
 'negative': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an empathetic customer service representative.\nSummarize the following negative feedback and generate an apology 

In [67]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos] # this is what we'll pass to the template
destinations_str = "\n".join(destinations)

destinations, destinations_str

(['positive: Good for processing positive feedback',
  'negative: Good for processing negative feedback',
  'neutral: Good for processing neutral feedback'],
 'positive: Good for processing positive feedback\nnegative: Good for processing negative feedback\nneutral: Good for processing neutral feedback')

In [68]:

default_prompt = ChatPromptTemplate.from_template("reply to this {input} gracefully")

default_chain = LLMChain(llm=llm, prompt=default_prompt)
# This chain acts as a fallback when the router is unable to categorize the input. It generates a generic response.

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ original input
}}}}
```

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
template=router_template,
input_variables=["input"],
output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [70]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [71]:
chain.run("I love the new update! The app is much faster and the design looks great.")



> Entering new MultiPromptChain chain...
positive: {'input': 'I love the new update! The app is much faster and the design looks great.'}
> Finished chain.


'Here\'s a summary of the feedback and a thank-you response:\n\n**Summary:** The customer is extremely satisfied with the new update to the app, praising its improved speed and design.\n\n**Thank-you response:**\n\n"Thank you so much for your kind words about our latest app update! We\'re thrilled to hear that you\'re enjoying the improved speed and design. Our team worked hard to bring you this update, and it\'s wonderful to know that it\'s making a positive impact on your experience. We\'re grateful for customers like you who take the time to share their feedback with us. If you have any other questions or need assistance with anything else, please don\'t hesitate to reach out. Thank you again for your support!"'

In [72]:
## Memory

In [74]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = ChatTogether(
    api_key=os.getenv("TOGETHER_API_KEY"),
    temperature= 0.2,
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    max_tokens=200
)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm = llm,
    memory = memory
)

In [75]:
conversation.predict(input = "Top challenge for Indian Government.")

"The top challenge for the Indian government is a multifaceted issue, and I'd be happy to provide some insights. According to various reports and studies, one of the most pressing challenges facing the Indian government is the issue of poverty and income inequality. As of 2022, approximately 220 million people in India live below the poverty line, which is roughly 17% of the country's population. This is despite India's rapid economic growth over the past few decades.\n\nAnother significant challenge is the issue of corruption, which is a major obstacle to good governance and economic development. The Indian government has implemented various initiatives to combat corruption, such as the Goods and Services Tax (GST) and the Jan Dhan Yojana scheme, which aims to provide financial inclusion to the poor.\n\nAdditionally, the Indian government is also grappling with the issue of climate change, which poses a significant threat to the country's agricultural sector, which is a major contribu

In [76]:
conversation.predict(input="Give me just top 2 and keep them consize")

"Based on my analysis, the top 2 challenges for the Indian government are:\n\n1. **Poverty and income inequality**: Approximately 220 million people in India live below the poverty line, which is roughly 17% of the country's population.\n2. **Corruption**: A major obstacle to good governance and economic development, with various initiatives implemented to combat it, such as the Goods and Services Tax (GST) and the Jan Dhan Yojana scheme."

In [79]:
print(conversation.predict(input="What you think could be the solution to this ?"))

A question that gets to the heart of the matter! Based on my analysis of various studies and reports, I'd like to propose some potential solutions to address the top 2 challenges facing the Indian government:

**Poverty and income inequality**:

1. **Targeted social welfare programs**: Implementing programs that provide direct financial assistance to the most vulnerable populations, such as the elderly, disabled, and those living in extreme poverty.
2. **Education and skill development**: Investing in education and vocational training to equip people with skills that are in demand in the job market, thereby increasing their employability and earning potential.

**Corruption**:

1. **Strengthening institutions**: Ensuring that institutions such as the judiciary, police, and anti-corruption agencies are independent, effective, and free from corruption.
2. **Digitalization and transparency**: Implementing digital platforms and tools to increase transparency and accountability in governmen

In [81]:
print(memory.buffer)

Human: Top challenge for Indian Government.
AI: The top challenge for the Indian government is a multifaceted issue, and I'd be happy to provide some insights. According to various reports and studies, one of the most pressing challenges facing the Indian government is the issue of poverty and income inequality. As of 2022, approximately 220 million people in India live below the poverty line, which is roughly 17% of the country's population. This is despite India's rapid economic growth over the past few decades.

Another significant challenge is the issue of corruption, which is a major obstacle to good governance and economic development. The Indian government has implemented various initiatives to combat corruption, such as the Goods and Services Tax (GST) and the Jan Dhan Yojana scheme, which aims to provide financial inclusion to the poor.

Additionally, the Indian government is also grappling with the issue of climate change, which poses a significant threat to the country's agr

In [82]:
print(memory.load_memory_variables({}))

{'history': "Human: Top challenge for Indian Government.\nAI: The top challenge for the Indian government is a multifaceted issue, and I'd be happy to provide some insights. According to various reports and studies, one of the most pressing challenges facing the Indian government is the issue of poverty and income inequality. As of 2022, approximately 220 million people in India live below the poverty line, which is roughly 17% of the country's population. This is despite India's rapid economic growth over the past few decades.\n\nAnother significant challenge is the issue of corruption, which is a major obstacle to good governance and economic development. The Indian government has implemented various initiatives to combat corruption, such as the Goods and Services Tax (GST) and the Jan Dhan Yojana scheme, which aims to provide financial inclusion to the poor.\n\nAdditionally, the Indian government is also grappling with the issue of climate change, which poses a significant threat to

In [83]:
# Coversation Buffer Window Memory

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.load_memory_variables({})


{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [89]:
# Conversation Token Based Buffer Memory
# ! pip install tiktoken

from langchain.memory import ConversationTokenBufferMemory

# llm = ChatTogether(
#     api_key=os.getenv("TOGETHER_API_KEY"),
#     temperature= 0.2,
#     model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
#     max_tokens=200
# )

llm = AzureChatOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-03-15-preview",
    temperature = 0.2, 
    model = "gpt-4o-mini"
    )

memory = ConversationTokenBufferMemory(llm=llm, 
                                       max_token_limit=50)

memory.save_context({"input": "Hello i am Suraaj"},
                    {"output": "i am an ai assistant"})

memory.save_context({"input": "how are you ai assistant"},
                    {"output": "i am fine, that you. how are you Suraaj"})

memory.save_context({"input": "working on understanding langchian"},
                    {"output": "i can help you in understanding langchain"})

memory.load_memory_variables({})

{'history': 'AI: i am fine, that you. how are you Suraaj\nHuman: working on understanding langchian\nAI: i can help you in understanding langchain'}

In [90]:
# Conversation Summary Memory

from langchain.memory import ConversationSummaryBufferMemory

# we have created a long string with lots of information
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."


memory = ConversationSummaryBufferMemory(llm=llm, 
                                         max_token_limit=100)

memory.save_context({"input": "Hello"}, {"output": "What's up"})

memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

In [91]:
print(memory.load_memory_variables({})["history"])

System: The human greets the AI, and they exchange casual pleasantries. The human then asks about the day's schedule.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
